In [1]:
from carbon import CarbonSimulatorUI, P, __version__, __date__
print(f"Carbon v{__version__} ({__date__})")
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSimulatorUI))

Carbon v2.0-beta2 (8/Dec/2022)
CarbonSimulatorUI v2.2 (8/Dec/2022)


# Carbon Simulation - Test 43 - Single Orders
## Single orders

### add_order as shortcut for add_strategy

In [2]:
Sim = CarbonSimulatorUI(pair="ETH/USDC", raiseonerror=True, exclude_future=False)
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETH/USDC', mm='exact', xf=False)

#### Base token (ETH)

In [3]:
r = Sim.add_order("ETH", 10, 2000, 3000)["orders"]
assert len(r) == 2
r

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,0


In [4]:
rr=r.query("tkn=='ETH'")
assert rr.iloc[0]["y"] == 10
assert rr.iloc[0]["disabled"] == False
assert rr.iloc[0]["p_start"] == 2000
ix = rr.iloc[0]["id"]
lix = rr.iloc[0]["lid"]
print(f"id={ix} lid={lix}")
rr

id=0 lid=1


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,1


In [5]:
rr=r.query("tkn=='USDC'")
assert rr.iloc[0]["y"] == 0
assert not rr.iloc[0]["y"] is None
assert rr.iloc[0]["disabled"] == True
assert rr.iloc[0]["p_start"] is None
ix2 = rr.iloc[0]["id"]
lix2 = rr.iloc[0]["lid"]
print(f"id={ix2} lid={lix2}")
assert ix == lix2
assert lix == ix2
rr

id=1 lid=0


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
1,1,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,0


#### Quote token (USDC)

In [6]:
r = Sim.add_order("USDC", 1000, 1000, 750)["orders"]
assert len(r) == 2
r

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
2,2,ETHUSDC,USDC,1000.0,1000.0,USDC,False,1000.0,750.0,1000.0,USDC per ETH,3
3,3,ETHUSDC,ETH,0.0,0.0,ETH,True,None,None,None,USDC per ETH,2


In [7]:
rr=r.query("tkn=='USDC'")
assert rr.iloc[0]["y"] == 1000
assert rr.iloc[0]["disabled"] == False
assert rr.iloc[0]["p_start"] == 1000
rr

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
2,2,ETHUSDC,USDC,1000.0,1000.0,USDC,False,1000.0,750.0,1000.0,USDC per ETH,3


In [8]:
rr=r.query("tkn=='ETH'")
assert rr.iloc[0]["y"] == 0
assert not rr.iloc[0]["y"] is None
assert rr.iloc[0]["disabled"] == True
assert rr.iloc[0]["p_start"] is None
rr

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
3,3,ETHUSDC,ETH,0.0,0.0,ETH,True,None,None,None,USDC per ETH,2


### _add_replace_single_order

this is a private function that allows to place single orders; this is an advanced feature and the API may change over time

#### Ensure it only works with future enabled

In [9]:
try:
    SimNF = CarbonSimulatorUI(pair="ETH/USDC", raiseonerror=True)
    SimNF._add_replace_single_order("ETH", 20, 2000, 3000)
except CarbonSimulatorUI.ExcludedFutureFunctionality as e:
    print(e)
    assert str(e) == "Feature disabled (us `exclude_future = False` to enable)"

Feature disabled (us `exclude_future = False` to enable)


#### Generating new orders (use with care; money disappears)

In [10]:
r = Sim._add_replace_single_order("ETH", 20, 2000, 3000)
ro=r["orders"]
print(f"id={r['id']}, lid={r['lid']}")
assert ro.iloc[0]["id"] == r['id']
assert ro.iloc[0]["lid"] == r['lid']
ro

id=4, lid=4


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
4,4,ETHUSDC,ETH,20.0,20.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,4


In [11]:
r = Sim._add_replace_single_order("USDC", None, 1000, 750)
ro=r["orders"]
print(f"id={r['id']}, lid={r['lid']}")
assert ro.iloc[0]["id"] == r['id']
assert ro.iloc[0]["lid"] == r['lid']
ro

id=5, lid=5


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
5,5,ETHUSDC,USDC,0.0,0.0,USDC,False,1000.0,750.0,1000.0,USDC per ETH,5


In [12]:
r = Sim._add_replace_single_order("ETH", 20, None, None)
ro=r["orders"]
print(f"id={r['id']}, lid={r['lid']}")
assert ro.iloc[0]["id"] == r['id']
assert ro.iloc[0]["lid"] == r['lid']
ro

id=6, lid=6


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
6,6,ETHUSDC,ETH,20.0,20.0,ETH,True,None,None,None,USDC per ETH,6


In [13]:
r = Sim._add_replace_single_order("USDC")
ro=r["orders"]
print(f"id={r['id']}, lid={r['lid']}")
assert ro.iloc[0]["id"] == r['id']
assert ro.iloc[0]["lid"] == r['lid']
ro

id=7, lid=7


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
7,7,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,7


In [14]:
r = Sim.state()["orders"]
assert len(r) == 8
r

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,0
2,2,ETHUSDC,USDC,1000.0,1000.0,USDC,False,1000.0,750.0,1000.0,USDC per ETH,3
3,3,ETHUSDC,ETH,0.0,0.0,ETH,True,None,None,None,USDC per ETH,2
4,4,ETHUSDC,ETH,20.0,20.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,4
5,5,ETHUSDC,USDC,0.0,0.0,USDC,False,1000.0,750.0,1000.0,USDC per ETH,5
6,6,ETHUSDC,ETH,20.0,20.0,ETH,True,None,None,None,USDC per ETH,6
7,7,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,7


#### Replacing existing orders
note: this completely replaces those orders; there are no checks whether the curve and token are the same

In [15]:
r = Sim._add_replace_single_order("ETH", 21, 2010, 3010, oid=4, lid=5)
ro=r["orders"]
ro

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
4,4,ETHUSDC,ETH,21.0,21.0,ETH,False,2010.0,3010.0,2010.0,USDC per ETH,5


In [16]:
r = Sim._add_replace_single_order("USDC", 1100, 1001, 751, oid=5, lid=4)
ro=r["orders"]
ro

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
5,5,ETHUSDC,USDC,1100.0,1100.0,USDC,False,1001.0,751.0,1001.0,USDC per ETH,4


In [17]:
r = Sim.state()["orders"]
assert len(r) == 8
r

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,0
2,2,ETHUSDC,USDC,1000.0,1000.0,USDC,False,1000.0,750.0,1000.0,USDC per ETH,3
3,3,ETHUSDC,ETH,0.0,0.0,ETH,True,None,None,None,USDC per ETH,2
4,4,ETHUSDC,ETH,21.0,21.0,ETH,False,2010.0,3010.0,2010.0,USDC per ETH,5
5,5,ETHUSDC,USDC,1100.0,1100.0,USDC,False,1001.0,751.0,1001.0,USDC per ETH,4
6,6,ETHUSDC,ETH,20.0,20.0,ETH,True,None,None,None,USDC per ETH,6
7,7,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,7


In [18]:
rr = r.query("id==5 or id==4")
assert len(rr) == 2
assert list(rr["id"]) == [4,5]
assert list(rr["lid"]) == [5,4]
rr

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
4,4,ETHUSDC,ETH,21.0,21.0,ETH,False,2010.0,3010.0,2010.0,USDC per ETH,5
5,5,ETHUSDC,USDC,1100.0,1100.0,USDC,False,1001.0,751.0,1001.0,USDC per ETH,4


#### Generating a collection account
We here link multiple positions to a single USD account; **note that this is _not_ functionality that the MVP would allow**

In [19]:
Sim = CarbonSimulatorUI(pair="ETH/USDC", raiseonerror=True, exclude_future=False)
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETH/USDC', mm='exact', xf=False)

Setting up the collection account in USDC

In [20]:
r = Sim._add_replace_single_order("USDC")
coid = r["id"]
print(f"Collection order id coid={coid}")
r["orders"]

Collection order id coid=0


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,0


Setting up an account selling ETH for USDC

In [21]:
r = Sim._add_replace_single_order("ETH", 10, 2000, 2000, lid=coid)
r["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
1,1,ETHUSDC,ETH,10.0,10.0,ETH,False,2000.0,2000.0,2000.0,USDC per ETH,0


Setting up an account selling WBTC for USDC

In [22]:
r = Sim._add_replace_single_order("WBTC", 1, 10000, 10000, pair="WBTC/USDC", lid=coid)
r["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
2,2,WBTCUSDC,WBTC,1.0,1.0,WBTC,False,10000.0,10000.0,10000.0,USDC per WBTC,0


Setting up an account selling BNT for USDC

In [23]:
r = Sim._add_replace_single_order("BNT", 100000, 2, 2, pair="BNT/USDC", lid=coid)
r["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
3,3,BNTUSDC,BNT,100000.0,100000.0,BNT,False,2.0,2.0,2.0,USDC per BNT,0


In [24]:
r = Sim.state()["orders"]
assert len(r) == 4
assert list(r["lid"]) == [coid]*4
assert len(set(r["id"])) == 4
r

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,0
1,1,ETHUSDC,ETH,10.0,10.0,ETH,False,2000.0,2000.0,2000.0,USDC per ETH,0
2,2,WBTCUSDC,WBTC,1.0,1.0,WBTC,False,10000.0,10000.0,10000.0,USDC per WBTC,0
3,3,BNTUSDC,BNT,100000.0,100000.0,BNT,False,2.0,2.0,2.0,USDC per BNT,0


In [25]:
Sim.amm_sells("ETH", 5)
r=Sim.state()["trades"]
r

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,5.0,ETH,10000.0,USDC,ETHUSDC,1,1,2000.0,USDC per ETH
0,0,0,A,AMM sells 5ETH buys 10000USDC,True,True,None,5.0,ETH,10000.0,USDC,ETHUSDC,[1],1,2000.0,USDC per ETH


In [26]:
r=Sim.state()["orders"]
r0=r.query(f"id=={coid}")
assert int(r0["y"]) == 5*2000
r

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,10000.0,10000.0,USDC,True,None,None,None,USDC per ETH,0
1,1,ETHUSDC,ETH,10.0,5.0,ETH,False,2000.0,2000.0,2000.0,USDC per ETH,0
2,2,WBTCUSDC,WBTC,1.0,1.0,WBTC,False,10000.0,10000.0,10000.0,USDC per WBTC,0
3,3,BNTUSDC,BNT,100000.0,100000.0,BNT,False,2.0,2.0,2.0,USDC per BNT,0


In [27]:
Sim.amm_sells("WBTC", 0.5, pair="WBTC/USDC")
Sim.amm_sells("BNT", 50000, pair="BNT/USDC")
r=Sim.state()["trades"]
r

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,5.0,ETH,10000.0,USDC,ETHUSDC,1,1,2000.0,USDC per ETH
0,0,0,A,AMM sells 5ETH buys 10000USDC,True,True,None,5.0,ETH,10000.0,USDC,ETHUSDC,[1],1,2000.0,USDC per ETH
0,1.0,1,0,route #2,False,True,None,0.5,WBTC,5000.0,USDC,WBTCUSDC,2,1,10000.0,USDC per WBTC
0,1,1,A,AMM sells 0WBTC buys 5000USDC,True,True,None,0.5,WBTC,5000.0,USDC,WBTCUSDC,[2],1,10000.0,USDC per WBTC
0,2.0,2,0,route #3,False,True,None,50000.0,BNT,100000.0,USDC,BNTUSDC,3,1,2.0,USDC per BNT
0,2,2,A,AMM sells 50000BNT buys 100000USDC,True,True,None,50000.0,BNT,100000.0,USDC,BNTUSDC,[3],1,2.0,USDC per BNT


In [28]:
r=Sim.state()["orders"]
r0=r.query(f"id=={coid}")
print(int(r0["y"]), 5*2000 + 0.5 * 10000 + 50000*2)
assert int(r0["y"]) == int(5*2000 + 0.5 * 10000 + 50000*2)
r

115000 115000.0


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,115000.0,115000.0,USDC,True,None,None,None,USDC per ETH,0
1,1,ETHUSDC,ETH,10.0,5.0,ETH,False,2000.0,2000.0,2000.0,USDC per ETH,0
2,2,WBTCUSDC,WBTC,1.0,0.5,WBTC,False,10000.0,10000.0,10000.0,USDC per WBTC,0
3,3,BNTUSDC,BNT,100000.0,50000.0,BNT,False,2.0,2.0,2.0,USDC per BNT,0
